# 아래는 여러 툴을 사용하며 다른 시스템과 상호작용하는 에이전트를 다룹니다.

In [1]:
%pip install --upgrade --quiet langchain-openai tavily-python

# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv

dotenv.load_dotenv('../dot.env')

import os
import getpass

# 주어진 환경 변수가 설정되어 있지 않다면 사용자에게 입력을 요청하여 설정합니다.
def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

# LangSmith 추적 기능을 활성화합니다. (선택적)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "QA_SQL_CSV_Toolusage"

Note: you may need to restart the kernel to use updated packages.


# LLM & Tool 정의

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

tools = [TavilySearchResults(max_results=1)]

# Choose the LLM that will drive the agent
# Only certain models support this
chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Adapted from https://smith.langchain.com/hub/hwchase17/openai-tools-agent
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. You may not need to use tools for every query - the user may just want to chat!",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [4]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(chat, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [5]:
from langchain_core.messages import HumanMessage

agent_executor.invoke({"messages": [HumanMessage(content="I'm Nemo!")]})



> Entering new AgentExecutor chain...
Hi Nemo! It's great to meet you. How can I assist you today?

> Finished chain.


{'messages': [HumanMessage(content="I'm Nemo!")],
 'output': "Hi Nemo! It's great to meet you. How can I assist you today?"}

In [8]:
from langchain_core.messages import HumanMessage

agent_executor.invoke({"messages": [
    HumanMessage(
    content="What is the current conservation status of the Great Barrier Reef?")
    ]
    }
)




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'current conservation status of the Great Barrier Reef'}`


[{'url': 'https://www.cnn.com/2022/08/04/australia/great-barrier-reef-high-coral-report-australia-climate-intl-hnk/index.html', 'content': 'CNN values your feedback\nParts of Great Barrier Reef record highest amount of coral in 36 years\nParts of the Great Barrier Reef have recorded their highest amount of coral cover since the Australian Institute of Marine Science (AIMS) began monitoring 36 years ago, according to a report published Thursday.\n “In Australia, to do our share, we really need to slash our emissions reductions by 75% by 2030 and that is to hold global warming to less than 1.5 degrees which is the critical threshold for the survival of coral reef as we know it,” Muddle told CNN.\n Many public figures, including “Aquaman” actor Jason Momoa and ocean explorer Philippe Cousteau, have called for UNESCO to add the natural 

{'messages': [HumanMessage(content='What is the current conservation status of the Great Barrier Reef?')],
 'output': 'The Great Barrier Reef has recorded its highest amount of coral cover since the Australian Institute of Marine Science (AIMS) began monitoring 36 years ago. This is a positive sign, indicating that the reef could still recover from mass bleaching and outbreaks of crown-of-thorns starfish that feed on coral. However, there are concerns about the survival of coral reefs due to global warming, and there have been calls for UNESCO to add the Great Barrier Reef to its "in danger" list in hopes of incentivizing action to save the reef.\n\nYou can read more about it [here](https://www.cnn.com/2022/08/04/australia/great-barrier-reef-high-coral-report-australia-climate-intl-hnk/index.html).'}

# massages에 직접 그간 대화를 넣을 수도 있습니다.

In [10]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "messages": [
            HumanMessage(content="I'm Nemo!"),
            AIMessage(content="Hello Nemo! How can I assist you today?"),
            HumanMessage(content="What is my name?"),
        ],
    }
)



> Entering new AgentExecutor chain...
Your name is Nemo! How can I assist you today, Nemo?

> Finished chain.


{'messages': [HumanMessage(content="I'm Nemo!"),
  AIMessage(content='Hello Nemo! How can I assist you today?'),
  HumanMessage(content='What is my name?')],
 'output': 'Your name is Nemo! How can I assist you today, Nemo?'}

 # 선호하는 경우, 에이전트 실행기를 RunnableWithMessageHistory 클래스에 래핑하여 내부적으로 히스토리 메시지를 관리할 수 있습니다.
 - 먼저, 래퍼가 히스토리로 저장할 입력 값을 구문 분석할 수 있도록 프롬프트를 약간 수정해야 합니다:


In [15]:
# Adapted from https://smith.langchain.com/hub/hwchase17/openai-tools-agent
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. You may not need to use tools for every query - the user may just want to chat!",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(chat, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [16]:
agent_executor.__dict__['agent']

RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant. You ma

# 에이전트 실행기에 여러 출력이 있기 때문에 래퍼를 초기화할 때 output_messages_key 속성을 설정해야 합니다:


In [17]:
# langchain.memory에서 ChatMessageHistory를 가져옵니다.
from langchain.memory import ChatMessageHistory

# langchain_core.runnables.history에서 RunnableWithMessageHistory를 가져옵니다.
from langchain_core.runnables.history import RunnableWithMessageHistory

# 데모용 휘발성 채팅 히스토리를 초기화합니다.
demo_ephemeral_chat_history_for_chain = ChatMessageHistory()

# RunnableWithMessageHistory를 사용하여 대화형 에이전트 실행기를 설정합니다.
conversational_agent_executor = RunnableWithMessageHistory(
    agent_executor,
    # 세션 ID를 받아서 데모용 휘발성 채팅 히스토리를 반환하는 람다 함수입니다.
    # 이 함수는 각 세션에 대해 동일한 히스토리 객체를 사용하도록 설정합니다.
    lambda session_id: demo_ephemeral_chat_history_for_chain,
    
    # 입력 메시지의 키를 "input"으로 설정합니다.
    input_messages_key="input",
    
    # 출력 메시지의 키를 "output"으로 설정합니다.
    output_messages_key="output",
    
    # 히스토리 메시지의 키를 "chat_history"로 설정합니다.
    history_messages_key="chat_history",
)

In [20]:
conversational_agent_executor.__dict__

{'name': None,
 'bound': RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
   chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
 }), config={'run_name': 'insert_history'})
 | RunnableBinding(bound=AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
     agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
   })
   | ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMess

In [18]:
conversational_agent_executor.invoke(
    {
        "input": "I'm Nemo!",
    },
    {"configurable": {"session_id": "unused"}},
)

Parent run 37252891-8ec6-4505-abc5-5409099d1153 not found for run b4f34f88-8c02-49f9-b468-d68564df1b9a. Treating as a root run.




> Entering new AgentExecutor chain...
Hi Nemo! It's great to meet you. How can I assist you today?

> Finished chain.


{'input': "I'm Nemo!",
 'chat_history': [],
 'output': "Hi Nemo! It's great to meet you. How can I assist you today?"}

In [19]:
conversational_agent_executor.invoke(
    {
        "input": "What is my name?",
    },
    {"configurable": {"session_id": "unused"}},
)

Parent run d04875aa-37f9-45f8-83cc-49e8ce5cd230 not found for run b4a6a921-44f4-48f4-9fcf-16b2a4f79760. Treating as a root run.




> Entering new AgentExecutor chain...
Your name is Nemo! How can I assist you today, Nemo?

> Finished chain.


{'input': 'What is my name?',
 'chat_history': [HumanMessage(content="I'm Nemo!"),
  AIMessage(content="Hi Nemo! It's great to meet you. How can I assist you today?")],
 'output': 'Your name is Nemo! How can I assist you today, Nemo?'}

In [21]:
conversational_agent_executor.invoke(
    {
        "input": "What is the current conservation status of the Great Barrier Reef?",
    },
    {"configurable": {"session_id": "unused"}},
)

Parent run 64695b14-79da-44b2-9a4b-27bf075dd95a not found for run bfb80aa8-05b9-4123-b935-1b25f6737c3d. Treating as a root run.




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'current conservation status of the Great Barrier Reef'}`


[{'url': 'https://www.cnn.com/2022/08/04/australia/great-barrier-reef-high-coral-report-australia-climate-intl-hnk/index.html', 'content': 'CNN values your feedback\nParts of Great Barrier Reef record highest amount of coral in 36 years\nParts of the Great Barrier Reef have recorded their highest amount of coral cover since the Australian Institute of Marine Science (AIMS) began monitoring 36 years ago, according to a report published Thursday.\n “In Australia, to do our share, we really need to slash our emissions reductions by 75% by 2030 and that is to hold global warming to less than 1.5 degrees which is the critical threshold for the survival of coral reef as we know it,” Muddle told CNN.\n Many public figures, including “Aquaman” actor Jason Momoa and ocean explorer Philippe Cousteau, have called for UNESCO to add the natural 

{'input': 'What is the current conservation status of the Great Barrier Reef?',
 'chat_history': [HumanMessage(content="I'm Nemo!"),
  AIMessage(content="Hi Nemo! It's great to meet you. How can I assist you today?"),
  HumanMessage(content='What is my name?'),
  AIMessage(content='Your name is Nemo! How can I assist you today, Nemo?')],
 'output': 'The current conservation status of the Great Barrier Reef is a topic of concern. According to a report published by the Australian Institute of Marine Science (AIMS), parts of the Great Barrier Reef have recorded their highest amount of coral cover in 36 years. This is seen as a positive sign, indicating that the reef could still recover from mass bleaching and outbreaks of crown-of-thorns starfish that feed on coral. However, there are ongoing calls from public figures to add the Great Barrier Reef to UNESCO\'s "in danger" list in hopes that it would incentivize action to save the reef.\n\nIf you\'d like to know more details, feel free to 